In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import pandas as pd
full_data = pd.read_csv("../../data/processed/historical/train/lcl_data.csv")
df_25K = full_data.sample(25000, random_state=0)
df_25K.to_csv("../../data/processed/historical/train/lcl_data_25K.csv", index=False)

# Load Data

In [15]:
import torch
import numpy as np
import random
RANDOM_STATE = 0
torch.manual_seed(RANDOM_STATE)
torch.use_deterministic_algorithms(True)
g = torch.Generator()
g.manual_seed(RANDOM_STATE)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [16]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data_25K.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=25000, n_samples=25000)
dm.setup()

In [17]:
import torch
from opensynth.models.faraday import FaradayVAE
vae_model = torch.load("vae_model.pt")
vae_model.eval()

/var/folders/w2/2x1gj41j1xb5z153q5m_4rn80000gn/T/ipykernel_47063/2311655816.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae_model = torch.load("vae_model.pt")


FaradayVAE(
  (encoder): Encoder(
    (encoder_layers): Sequential(
      (0): Linear(in_features=50, out_features=512, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): GELU(approximate='none')
      (4): Linear(in_features=256, out_features=128, bias=True)
      (5): GELU(approximate='none')
      (6): Linear(in_features=128, out_features=64, bias=True)
      (7): GELU(approximate='none')
      (8): Linear(in_features=64, out_features=32, bias=True)
      (9): GELU(approximate='none')
      (10): Linear(in_features=32, out_features=16, bias=True)
    )
  )
  (decoder): Decoder(
    (latent): Linear(in_features=18, out_features=16, bias=True)
    (latent_activations): GELU(approximate='none')
    (decoder_layers): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): GELU(approximate='no

In [18]:
from opensynth.models.faraday.gaussian_mixture.prepare_gmm_input import encode_data_for_gmm

next_batch = next(iter(dm.train_dataloader()))
input_tensor = encode_data_for_gmm(data=next_batch, vae_module=vae_model)
input_data = input_tensor.detach().numpy()
n_samples = len(input_tensor)

In [66]:
N_COMPONENTS = 250
REG_COVAR = 1e-4
EPOCHS = 25
IDX = 0
CONVERGENCE_TOL = 1e-2


In [12]:
input_tensor[0][0]

tensor(0.0195, grad_fn=<SelectBackward0>)

# Init GMM

In [20]:
from opensynth.models.faraday.new_gmm import gmm_utils

labels_, means_, responsibilities_ = gmm_utils.initialise_centroids(
        X=input_data, n_components=N_COMPONENTS
    )
print(labels_.dtype, responsibilities_.dtype, means_.dtype)

torch.float32 torch.float32 torch.float32


In [21]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params

gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
print(gmm_init_params["precision_cholesky"][IDX][0][0])
print(gmm_init_params["weights"].sum())

tensor(4.1672)
tensor(1.)


In [22]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params, training_loop
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureModel


gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
torch_gmm = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
torch_gmm.initialise(gmm_init_params)
trained_model = training_loop(model=torch_gmm, data=input_tensor, max_iter=EPOCHS)

100%|██████████| 25/25 [00:08<00:00,  2.89it/s]


# SK Learn GMM Manual

In [23]:
import numpy as np
from scipy.special import logsumexp
from scipy import linalg

def _estimate_gaussian_parameters(X, resp, reg_covar=REG_COVAR):
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]
    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[:: n_features + 1] += reg_covar
    return nk, means, covariances

def _compute_precision_cholesky(covariances):
    estimate_precision_error_message = (
        "Fitting the mixture model failed because some components have "
        "ill-defined empirical covariance (for instance caused by singleton "
        "or collapsed samples). Try to decrease the number of components, "
        "or increase reg_covar."
    )

    n_components, n_features, _ = covariances.shape
    precisions_chol = np.empty((n_components, n_features, n_features))
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
        except linalg.LinAlgError:
            raise ValueError(estimate_precision_error_message)
        precisions_chol[k] = linalg.solve_triangular(
            cov_chol, np.eye(n_features), lower=True
        ).T
    return precisions_chol

def _compute_log_det_cholesky(matrix_chol, n_features):
    n_components, _, _ = matrix_chol.shape
    log_det_chol = np.sum(
        np.log(matrix_chol.reshape(n_components, -1)[:, :: n_features + 1]), 1
    )
    return log_det_chol

def _estimate_log_gaussian_prob(X, means, precisions_chol):
    n_samples, n_features = X.shape
    n_components, _ = means.shape

    log_det = _compute_log_det_cholesky(precisions_chol, n_features)

    log_prob = np.empty((n_samples, n_components))
    for k, (mu, prec_chol) in enumerate(zip(means, precisions_chol)):
        y = np.dot(X, prec_chol) - np.dot(mu, prec_chol)
        log_prob[:, k] = np.sum(np.square(y), axis=1)
    return -0.5 * (n_features * np.log(2 * np.pi) + log_prob) + log_det

def _estimate_log_weights(weights):
        return np.log(weights)

def _estimate_weighted_log_prob(X, means, precisions_chol, weights):
        return _estimate_log_gaussian_prob(X, means, precisions_chol) + _estimate_log_weights(weights)


def _estimate_log_prob_resp(X, means, precisions_chol, weights):
    weighted_log_prob = _estimate_weighted_log_prob(X, means, precisions_chol, weights)
    log_prob_norm = logsumexp(weighted_log_prob, axis=1)
    with np.errstate(under="ignore"):
        log_resp = weighted_log_prob - log_prob_norm[:, np.newaxis]
    return log_prob_norm, log_resp

def _e_step(X,means, precisions_chol, weights):
    log_prob_norm, log_resp = _estimate_log_prob_resp(X, means, precisions_chol, weights)
    return np.mean(log_prob_norm), log_resp

def _m_step(X, log_reponsibilities, reg_covar=REG_COVAR):

    weights_, means_, covariances_ = _estimate_gaussian_parameters(X,np.exp(log_reponsibilities),reg_covar=reg_covar)

    precision_cholesky_ = _compute_precision_cholesky(covariances=covariances_)

    return precision_cholesky_, weights_, means_

In [68]:
means = gmm_init_params["means"].detach().numpy()
weights = gmm_init_params["weights"].detach().numpy()
prec_chol = gmm_init_params["precision_cholesky"].detach().numpy()

print(f"Initial prec chol: {prec_chol[IDX][0][0]}. Initial mean: {means[IDX][0]}")

converged = False
lower_bound = -np.inf

for i in range(EPOCHS):
    prev_lower_bound = lower_bound

    print(f"Old Prec Chol: {prec_chol[IDX][0][0]}. Old means: {means[IDX][0]}")
    log_prob, log_resp = _e_step(input_data, means, prec_chol, weights)
    prec_chol, weights, means = _m_step(input_data, log_resp)

    print(f"New prec chol: {prec_chol[IDX][0][0]}. New means: {means[IDX][0]}")

    # Converegence
    lower_bound = log_prob
    change = abs(lower_bound - prev_lower_bound)
    print(f"Change: {change}")
    if change < CONVERGENCE_TOL:
        converged = True
        break

print(f'Converged: {converged}. Number of iterations: {i}')

Initial prec chol: 4.156552314758301. Initial mean: 0.18224604427814484
Old Prec Chol: 4.156552314758301. Old means: 0.18224604427814484
New prec chol: 4.21163155249816. New means: 0.16120435878158323
Change: inf
Old Prec Chol: 4.21163155249816. Old means: 0.16120435878158323
New prec chol: 4.284132502748568. New means: 0.13693155041770586
Change: 10.723142805224304
Old Prec Chol: 4.284132502748568. Old means: 0.13693155041770586
New prec chol: 4.144575468926109. New means: 0.11688720733005029
Change: 0.23741509353996193
Old Prec Chol: 4.144575468926109. Old means: 0.11688720733005029
New prec chol: 3.8862033482301266. New means: 0.09808561390437934
Change: 0.15518131910796384
Old Prec Chol: 3.8862033482301266. Old means: 0.09808561390437934
New prec chol: 3.7472209039770306. New means: 0.08675231976610723
Change: 0.11684219461641554
Old Prec Chol: 3.7472209039770306. Old means: 0.08675231976610723
New prec chol: 3.6843574862228907. New means: 0.0776786422072779
Change: 0.0973880068637

# SK Learn GMM Epoch

In [43]:
from sklearn.mixture import GaussianMixture

init_weights = gmm_init_params["weights"]
init_means = gmm_init_params["means"]

skgmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type='full', tol=CONVERGENCE_TOL, max_iter=EPOCHS, random_state=0, means_init = init_means, weights_init=init_weights)
skgmm.fit(input_data)
skgmm_pred = skgmm.predict(input_data)

# Torch Lightning

In [26]:
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import LightningDataModule
class CustomDataset(Dataset):
    def __init__(self, data_tensor: torch.Tensor):
        self.data = data_tensor
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    
class CustomDataModule(LightningDataModule):
    def __init__(self, data_tensor: torch.Tensor, batch_size: int):
        super().__init__()
        self.data_tensor = data_tensor
        self.batch_size = batch_size
    def setup(self, stage=""):
        self.custom_ds = CustomDataset(self.data_tensor)
    def train_dataloader(self):
        return DataLoader(self.custom_ds, batch_size=self.batch_size, shuffle=False, generator=g, worker_init_fn=seed_worker)
    
custom_dm = CustomDataModule(data_tensor=input_tensor, batch_size=25000)
custom_dm.setup(stage="")

In [27]:

for i in range(5):
    print(next(iter(custom_dm.train_dataloader()))[0][0])

tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)


In [53]:
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureLightningModule
gmm_module = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
gmm_module.initialise(gmm_init_params)
gmm_lightning_module = GaussianMixtureLightningModule(
    gmm_module = gmm_module,
    vae_module = vae_model,
    num_components = gmm_module.num_components,
    num_features = gmm_module.num_features,
    reg_covar = gmm_module.reg_covar,
    convergence_tolerance = CONVERGENCE_TOL
)
trainer = pl.Trainer(max_epochs=EPOCHS, accelerator="cpu", deterministic=True )
trainer.fit(gmm_lightning_module, custom_dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name       | Type                 | Params | Mode 
------------------------------------------------------------
0 | gmm_module | GaussianMixtureModel | 0      | train
1 | vae_module | FaradayVAE           | 402 K  | eval 
------------------------------------------------------------
402 K     Trainable params
0         Non-trainable params
402 K     Total params
1.609     Total estimated model params size (MB)
1         Modules in train mode
32        Modules in eval mode


Epoch 18: 100%|██████████| 1/1 [00:00<00:00,  2.35it/s, v_num=4]


# Compare

In [54]:
IDX = 0

In [55]:
skgmm.means_[IDX]

array([-0.03409357, -1.60444181, -0.84525173,  1.52687141,  0.52739176,
       -0.08208853, -0.52802043,  3.27411926, -0.26092997, -1.67757124,
       -0.45017775, -1.3417439 , -2.34126148,  1.01930178,  1.13318752,
       -1.19118948,  7.24041631,  2.94375444])

In [56]:
trained_model.means[IDX]

tensor([-0.0496, -1.5120, -0.8428,  1.5137,  0.5313, -0.0923, -0.5077,  3.3233,
        -0.2645, -1.6030, -0.4354, -1.3286, -2.4643,  1.0428,  1.1234, -1.0958,
         7.2376,  2.8979])

In [57]:
means[IDX]

array([ 0.05927229, -1.82203529, -1.10582344,  1.86211818,  0.59518445,
       -0.0850846 , -0.86206195,  3.34695201, -0.32593068, -1.957371  ,
       -0.71917021, -1.64930341, -2.25145595,  0.90938136,  1.09867196,
       -1.39409819,  7.54327722,  2.87144065])

In [58]:
gmm_lightning_module.gmm_module.means[IDX]

tensor([-0.0291, -1.5898, -0.8670,  1.5469,  0.5635, -0.0816, -0.5269,  3.3206,
        -0.2614, -1.6546, -0.4404, -1.3841, -2.4280,  1.0509,  1.1529, -1.1667,
         7.2701,  2.8376])

In [59]:
gmm_init_params["means"][IDX]

tensor([ 0.1822, -2.7588, -1.0983,  1.7424,  0.2821, -0.0517, -0.9845,  3.2832,
        -0.2812, -2.2875, -0.6615, -1.5619, -1.2599,  0.9391,  0.9560, -2.0816,
         5.9167,  3.9500])

In [60]:
skgmm.precisions_cholesky_[IDX][0]

array([ 3.62810758,  2.5046845 ,  3.00098303, -0.97427171, -2.99772135,
       -5.10590263,  3.19043596, 12.39832528,  0.46417462, -7.09667743,
        3.30860913,  6.35795778, -5.38121506, -7.91289196,  4.75926963,
        4.36874473,  3.94478268,  6.27902463])

In [61]:
trained_model.precision_cholesky[IDX][0]

tensor([ 3.7405,  2.7770,  2.0497, -1.4382, -3.5489, -3.9533,  3.7853, 10.4361,
        -0.8158, -4.6887,  1.9838,  0.9060, -6.9250, -5.2341,  3.1970,  3.3125,
         6.2404,  4.4628])

In [62]:
prec_chol[IDX][0]

array([ 3.23311398,  3.00014493,  2.17134816, -2.10335049, -2.93972487,
       -4.50479356,  2.84537901, 12.56316785,  1.79440568, -4.35304584,
        2.79591458,  2.80417888, -5.89923265, -3.83837919,  1.841089  ,
        2.72552406,  4.37941362,  3.91518476])

In [63]:
gmm_lightning_module.gmm_module.precision_cholesky[IDX][0]

tensor([ 3.7507,  2.9416,  2.1470, -1.3012, -3.5715, -4.0825,  3.6712, 10.4498,
        -0.9893, -4.7431,  2.0277,  1.2090, -6.9990, -5.0247,  3.0595,  3.6701,
         5.6456,  4.3372])

In [64]:
gmm_init_params["precision_cholesky"][IDX][0]

tensor([ 4.1566,  1.6513,  2.5846, -2.7945, -3.7173, -4.3137,  2.0308,  6.6499,
        -0.3323, -2.4590, -0.2854,  3.9629, -2.3193, -2.7534, -1.5411, -0.7101,
         2.3265, -0.1992])